# Nested with-statements & exceptions

Any exception propagated from inner context-managers will be seen by outer context-managers.  Likewise if an inner context-manager swallows an exception, then it will not be seen by other ones.  Illustrate by creating a simple context manager that can be configured to either propagate or swallow exceptions:

In [ ]:
import contextlib

@contextlib.contextmanager
def propagater(name, propagate):
    try:
        yield
        print(name, 'exited normally.')
    except Exception:
        print(name, 'received an exception!')
        if propagate:
            raise

IN the above, if the `propagate` argument to `propagater` is `True`, it will propagate any exceptions that come from the body nested beneath it.  Otherwise, it will swallow those exceptions.

See how an inner context-manager can swallow exceptions so that an outer one never sees them:

In [ ]:
with propagater('outer', True), propagater('inner', False):
    raise TypeError('Cannot convert lead into gold.')

Likewise, the inner one can propagate them while the outer one swallows them:

In [ ]:
with propagater('outer', False), propagater('inner', True):
    raise TypeError('Cannot convert lead into gold.')

Since there is no report of an exception in this case, be assured that no exception escaped the with-statement.

## Do not pass a collection!

When using multiple context-managers with a single with-statment do not try to use a tuple or some other sequence of context-managers. This could result in a mysterious error message:

In [ ]:
import contextlib

@contextlib.contextmanager
def nest_test(name):
    print('Entering', name)
    yield
    print('Exiting', name) 

In [ ]:
with nest_test('outer'), nest_test('inner'):
    print('BODY')

In [ ]:
with(nest_test('a'), 
     nest_test('b')):
    pass

While the above errors seems to stat that the context-manager is missing an attribute, that is not the case.  The problem is that there is an attempt to pass a _tuple_ to the with-statement as a context-manager.  The with-statement does not try to unpack sequences; it simply tries to use what it is passed as a context-manager.  So the with-statement is looking for `__enter__()` on a tuple that it is getting passed, of course it fails with an `AttributeError` since tuples do not support the context manager protocol.  To remedy remove the parentheses and put everything back on one line:

In [ ]:
with nest_test('a'), nest_test('b'):
    pass

If there are several context-managers that need to be split over several lines, a line-continuation backslash must be used to put the statements on one _logical_ split over multiple _physical_ lines.  Example:

In [ ]:
with nest_test('a'), \
     nest_test('b'), \
     nest_test('c'):
    pass